# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響
- 比較 col selection: (Day_012_HW: numeric) V.S (Day_024_HW: object)


# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?
* 答：由例題的One hot encoding + linear regression組合score 為負，及使用linear都不過score:0.7，可知此模型使用線性迴歸影響頗大

* 註：Label Encoding: 小量且數值多 / Tree Model
  One Hot Encoding: 大量且特徵重要性高 / DL
  通常作法如上，house price 算是資料量大，但類別特徵重要性不大，我會選擇使用Label Encoding 找到最佳解


In [13]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor

# Load and recombine data
data_path = 'C:/Users/rich chang/Documents/GitHub/3rd-ML100Days/data/'
training_set = pd.read_csv(data_path + 'titanic_train.csv')
testing_set = pd.read_csv(data_path + 'titanic_test.csv')

Y_train = training_set['Survived']
training_set = training_set.drop(['PassengerId', 'Survived'] , axis=1)
testing_set = testing_set.drop(['PassengerId'] , axis=1)

dataset = pd.concat([training_set, testing_set])
dataset.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []

for dtype, feature in zip(dataset.dtypes, dataset.columns):
    if dtype == 'object':
        object_features.append(feature)
        
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
dataset = dataset[object_features]
dataset = dataset.fillna('None')
train_num = Y_train.shape[0]
dataset.head()


5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2(6種組合: 3regressions ,2encodings)
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)
* 答：Label Encoding有score 78，但One hot encoding 整個只剩40，影響更鉅。 
- 反倒Gradient tree boosting 相較linear regression, logistic regression 影響小的很多
- 作業 titanic 資料不算特多(400, 500筆)，但又有幾個特徵重要性高的類別特徵，需要都去試試，取最好的值


In [15]:
# Label encoding + linear regression
dataset_temp = pd.DataFrame()

for i in dataset.columns:
    dataset_temp[i] = LabelEncoder().fit_transform(dataset[i])

X_train = dataset_temp[:train_num]
estimator = LinearRegression()
start = time.time()

print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator, X_train, Y_train, cv=5).mean()}')
print(f'time : {time.time() - start} sec')


shape : (891, 5)
score : 0.33494273995838764
time : 0.04175400733947754 sec


In [16]:
# One hot encoding + linear regression
dataset_temp = pd.get_dummies(dataset)

X_train = dataset_temp[:train_num]
estimator = LinearRegression()
start = time.time()

print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator, X_train, Y_train, cv=5).mean()}')
print(f'time : {time.time() - start} sec')


shape : (891, 2429)
score : 0.4130754436483536
time : 1.3986051082611084 sec


In [17]:
# Label encoding + logistic regression
dataset_temp = pd.DataFrame()

for i in dataset.columns:
    dataset_temp[i] = LabelEncoder().fit_transform(dataset[i])

X_train = dataset_temp[:train_num]
estimator = LogisticRegression(solver='liblinear')
start = time.time()

print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator, X_train, Y_train, cv=5).mean()}')
print(f'time : {time.time() - start} sec')


shape : (891, 5)
score : 0.780004837244799
time : 0.05775308609008789 sec


In [18]:
# One hot encoding + logistic regression
dataset_temp = pd.get_dummies(dataset)

X_train = dataset_temp[:train_num]
estimator = LinearRegression()
start = time.time()

print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator, X_train, Y_train, cv=5).mean()}')
print(f'time : {time.time() - start} sec')


shape : (891, 2429)
score : 0.4130754436483536
time : 1.384765863418579 sec


In [19]:
# Label encoding + Gradient tree boosting
dataset_temp = pd.DataFrame()

for i in dataset.columns:
    dataset_temp[i] = LabelEncoder().fit_transform(dataset[i])
    
X_train = dataset_temp[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()
print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator, X_train, Y_train, cv=5).mean()}')
print(f'time : {time.time() - start} sec')


shape : (891, 5)
score : 0.37814706577577406
time : 0.3826112747192383 sec


In [20]:
# One hot encoding + Gradient tree boosting
dataset_temp = pd.get_dummies(dataset)
X_train = dataset_temp[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()
print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator, X_train, Y_train, cv=5).mean()}')
print(f'time : {time.time() - start} sec')


shape : (891, 2429)
score : 0.3772352131371055
time : 13.727847337722778 sec
